# Measurement of Loudspeaker to KEMAR on VariSphere Turntable

This notebook sketches how to measure binaural impulse responses with a KEMAR manikin. 

* Two sweeps are played, one for the loudspeaker and one for the analog feedback. 
* Three channels are recorded, two ears and the analog feedback.
* After each measurement, the turntable rotates by 1 deg.

In [ ]:
# Automatically reload the acoustics_hardware package if anything is changed
%load_ext autoreload
%autoreload 2

# Load requirements
import sys

import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
from IPython import display
from numpy.fft import rfft as fft, rfftfreq as fftfreq
from pathlib import Path
from time import sleep

import acoustics_hardware.serial

## Set up audio hardware

In [ ]:
print(acoustics_hardware.devices.AudioDevice.get_devices())

device_name = 'Orion 32'
fs = 48000

try:
    device = acoustics_hardware.devices.AudioDevice(
        name=device_name,
        fs=fs,
    )
except ValueError as e:
    sys.exit(e)

# 3 inputs (0..31) for 'Orion 32'
device.add_input(0)
device.add_input(1)
device.add_input(2)

# 2 inputs (0..31) for 'Orion 32'
device.add_output(0)
device.add_output(1)

amplitude_db = -20
sweep_duration = 3 # sec

amplitude_lin = 10**(amplitude_db/20)

# Signal is automatically played through the first 2 channels
generator = acoustics_hardware.generators.SweepGenerator(device=device,
    start_frequency=50, stop_frequency=20e3, duration=sweep_duration, repetitions=1, amplitude=amplitude_lin)

## Prepare functions for plotting

In [ ]:
# noinspection PyShadowingNames
def plot_data(data, fs):
    n_ch, n_sample = data.shape
    t_data = np.linspace(0, n_sample / fs, n_sample)
    freq_data = 20*np.log10(abs(fft(data)))
    f = fftfreq(n_sample, 1 / fs)
    
    fig, axes = plt.subplots(n_ch, 2, sharex='col', figsize=(12, 3 * n_ch))
    for ch in np.arange(n_ch): # individual channels
        # Time domain
        ax = axes[ch, 0]
        ax.plot(t_data, data[ch, :])
        ax.set_xlim(0, n_sample / fs)
        ax.legend([f'Channel {ch}'], loc='upper right')
        ax.grid()
        if ch < n_ch - 1:
            ax.set_xlabel('')
        else:
            ax.set_xlabel('t (s)')

        # Frequency domain
        y_max = np.ceil(freq_data[ch, :].max() / 5) * 5
        ax = axes[ch, 1]
        ax.semilogx(f, freq_data[ch, :])
        ax.set_xlim(30, fs/2)
        ax.set_ylim(y_max - 80, y_max)
        ax.legend([f'Channel {ch}'], loc='upper right')
        ax.grid()
        if ch < n_ch - 1:
            ax.set_xlabel('')
        else:
            ax.set_xlabel('f (Hz)')

## Test audio

Execute this cell to test if the audio setup works.

In [ ]:
# execute this cell to test if the audio setup works

file_name = 'file_test.mat'

device.start(timed=sweep_duration + 1, blocking=True)
device.reset()
data = device.get_input_data()


# Store the data
sio.savemat(file_name, {'recorded_signal': np.transpose(data), 'fs': fs})

# Show audio data preview
for ch in range(data.shape[0]):
    display.display(
        display.Markdown(f"<h3>Channel {ch}</h3>"),
        display.Audio(data[ch, :], rate=fs),
    )

# Plot data
plot_data(data, fs)

## Test VariSphere

Initialize VariSphere once and then use the second cell to move the turntable as many times as you want.

In [ ]:
vari = acoustics_hardware.serial.VariSphere(az_port='4001', el_port=None, ip='192.168.127.120')

In [ ]:
# Try different angles (in deg) here
#vari.move(az=0)
vari.move_blocking(az=0)

## Perform the measurement series

In [ ]:
# note that the folder "data" needs to exist
out_file_name = "data/kemar_azimuth_{:03d}_el_0.mat"

# Create output data path if it does not exist
Path(out_file_name).parent.mkdir(parents=True, exist_ok=True)

azis = np.arange(360)
#azis = np.arange(180, 181)
device.initialize()
for azi in azis:
    print(azi)
    vari.move_blocking(az=azi)
    
    # wait for the boom vibrations to settle
    sleep(1)
    
    #print("Taking a measurement.")
    
    # start recording
    device.start(timed=sweep_duration + 1, blocking=True)
    device.reset()
    data = device.get_input_data()
    
    # save the data
    sio.savemat(out_file_name.format(azi), {'recorded_signal': np.array(data).transpose(), 'fs': fs})
    
    # plot the data for sanity
    # plot_data_t(data)
    # plot_data_f(data, fs)

print('Measurement completed.')